### 발송 대상 파일 작성
1. 폴더 생성
    * 규칙 : 날짜 + 생성순서 - 구분자 '_'
    * 날짜 : YYYYMMDD
    * 생성순서 : 2자리 번호
    * 예) 20171005_02
2. 파일 생성
    * 가장 최신의 폴더에 저장
    * 규칙 : AUTH_서비스구분코드 + 실시간구분 + 날짜 + 노선 + 기간 + 임의숫자 - 구분자 '_'
    * 서비스구분코드 : FareReport(60 가격 트랜드), DailyReport(일일 가격 비교)
    * 실시간구분 : A(항공사), B(항공사사이트 이외 - 실시간)
    * 날짜 : YYYYMMDD
    * 노선 : 출발지 3자리코드+도착지 3자리코드
    * 기간 : 생성 데이터 기간 숫자
    * 임의숫자 : 타임스탬프
    * 예) 1_FareReport_A_20170910_GMPCJU_60_4143243.html
3. 기준 정보
    * 서비스구분코드 - [출발지,도착지,기간,사이트코드,생성함수]
    * 사이트코드 - 조회해올 사이트코드 리스트 [IP],[7C,LJ,ZE,...]
    * 생성함수 - 조건에 맞는 파일 생성용 함수

In [1]:
import copy
import json
import sqlite3
import pandas as pd
import datetime,time
import os
from common.util import save_raw_data, num_format, time_format, list_formatter, get_dirs, get_files, move_file
from common.web_util import make_grid_td, make_grid_th
from common.mail_service_util import email_sender, make_newletter_html
from common.log_util import log, logger_initialize
from common.env_variable import JOIN_DIR,JOIN_OK_DIR

logger_initialize('daily_mail_logger_setting.json')

In [2]:
REPORT_TEMPLATE_FILE = {
    'FareReport':'template/fare_linechart_monoton.html',
    'DailyReport':'template/daily_fare_report.html'
}

def init_mail_folder(parent_dir):
    ## html 폴더에서 폴더 리스트를 찾아 오늘 날짜의 폴더를 체크하여 폴더 생성
    ## YYYYMMDD_seq
    today = datetime.datetime.today().strftime("%Y%m%d")
    dirs = get_dirs(parent_dir,today)
    save_dir = "{}/{}_".format(parent_dir,today)
    if len(dirs) > 0:
        save_dir += str(int(dirs[-1].split("_")[-1])+1)

    else:
        save_dir += "1"
    os.mkdir(save_dir)
    return save_dir
def html_filename(code):
    # auth+서비스구분코드 + 실시간구분 + 날짜 + 노선 + 기간 + 임의숫자 - 구분자 '_'
    today = datetime.datetime.today().strftime("%Y%m%d")
    isairline = 'A' if code['isairline'] else 'B'
    return "{}_{}_{}_{}_{}{}_{}_{}.html".format(code['auth'],code['code'],isairline,today,
                                                code['dpt'],code['arr'],code['days'],int(time.time()))
    
## 사이트의 최근 스크랩날짜 얻기 위한 쿼리 생성
def latest_scrapdate_query(site_code):
    query = 'select max(scrap_date) from airfare_scraped_data where scrap_site = "{}"'.format(site_code)
    print("query : ",query)
    return query
    
def report_query(param):
    ## 기본 쿼리 작성
    param_ = {}
    param_['days'] = param.get('days','60') ## 기본 60일
    param_['dpt'] = param['dpt'] ## 출도착지가 없으면 에러
    param_['arr'] = param['arr']
    scrap_date = param.get('scrap_date')
    if scrap_date is None:
        scrap_date = "(select max(scrap_date) from airfare_scraped_data where scrap_site in ({scrap_site}) and dpt='{dpt}' and arr='{arr}')"
    else:
        scrap_date = "'{scrap_date}'"
        param_['scrap_date'] = param['scrap_date']
    sql = '''select airline, flt, dpt_date, dpt, arr, dpt_time, arr_time, fare, tax1, tax2, seat
        from airfare_scraped_data
        where scrap_date = '''
    sql = sql + scrap_date
    sql = sql + ''' and scrap_site in ({scrap_site})
        and dpt='{dpt}' and arr='{arr}'
        and dpt_date <= strftime('%Y%m%d','now','+{days} day')
        and airline in ({airline})'''
    param_['scrap_site'] = ','.join(["'"+c+"'" for c in param.get('sitecodes',[])])
    param_['airline'] = ','.join(["'"+c+"'" for c in param.get('airline',[])])
    query = sql.format(**param_)
    print("query : ",query)
    return query

## 60일 일기준 각 항공사 최소, 평균, 최대 가격 리포트 생성 함수
def fare_report_func(db,site_code,scrap_date=None):
    ## read list => make html => make file
    if scrap_date is None: #가장 최근 정보 읽어오기
        conn = sqlite3.connect(db)
        with conn:
            cur = conn.cursor()
            #cur.execute('select max(scrap_date) from airfare_scraped_data')
            cur.execute(latest_scrapdate_query(site_code['sitecodes'][0]))
            scrap_date = cur.fetchone()[0]

    param = copy.deepcopy(site_code);
    param['template'] = REPORT_TEMPLATE_FILE[site_code['code']]
    param['scrap_date'] = scrap_date
    query = report_query(param)
    query = "select dpt_date,airline,min(fare),avg(fare),max(fare) from(" +\
        query +\
        ") where fare != '' and fare > 0 group by airline,dpt_date order by airline,dpt_date"

    report_list = []
    conn = sqlite3.connect(db)
    with conn:
        cur = conn.cursor()
        cur.execute(query)
        rows = cur.fetchall()
        for row in rows:
            report_list.append([row[0], row[1], row[2], row[3], row[4]])
    if len(report_list) <= 0:
        return None
    raw_data = fare_report_html(report_list,param)
    return raw_data
    

## 템플릿 이용 HTML 파일 작성
def fare_report_html(report_list,param):#sitecodes,dpt,arr,scrap_date):
    ## 검색조건 만들기
    cond_html = "대상사이트 : {}<br>노선 : {} 60일<br>{} 스크랩핑 데이터".format('/'.join(param['sitecodes']),
                                                    param['dpt']+'-'+param['arr'],param['scrap_date'])
    #with open('template/fare_linechart_monoton.html',encoding='utf-8') as fp:
    with open(param['template'],encoding='utf-8') as fp:
        template = fp.read()
        chart_html = fare_report_chart(report_list)
        grid_html = fare_report_grid(report_list)
    return template.replace('{condition_data}',cond_html).replace('{chart_data}',chart_html).replace('{grid_data}',grid_html).replace('\n','')

## chart 데이터 작성
def fare_report_chart(report_list):
    chart_data = {}
    for td in report_list:
        fare_id = td[1]+'Min' 
        fare_values = {'date':td[0],'fares':int(td[2])}
        chart_data[fare_id]= chart_data.get(fare_id,[])
        chart_data[fare_id].append(fare_values)
        fare_id = td[1]+'Avg' 
        fare_values = {'date':td[0],'fares':int(td[3])}
        chart_data[fare_id]= chart_data.get(fare_id,[])
        chart_data[fare_id].append(fare_values)
        fare_id = td[1]+'Max' 
        fare_values = {'date':td[0],'fares':int(td[4])}
        chart_data[fare_id]= chart_data.get(fare_id,[])
        chart_data[fare_id].append(fare_values)
    chart_list = []
    for data_id,data_values in chart_data.items():
        chart_list.append(dict((('id',data_id),('values',data_values))))
    return json.dumps(chart_list)
## Grid 데이터 작성
def fare_report_grid(report_list):
    grid_html = ''
    tr_html = '<tr><td>{}</td><td>{}</td><td>{}</td><td>{}</td><td>{}</td></tr>'
    for td in report_list:
        grid_html += tr_html.format(td[0],td[1],int(td[2]),int(td[3]),int(td[4]))
    return grid_html

## 21일간 일별 플라이트별 최소 가격 비교 레포트 생성 함수
'''
    0. DataFrame 생성, fare 컬럼의 '' 값 0으로 보정
    1. 필터 : 항공사별, 날짜별, 플라이트별 최소값 데이터만 추출
        - 모든값이 '',0 이면 0
        - '',0 이외의 값중 최소값
    2. ZE 데이터 추출
    3. ZE 기준 경쟁 플라이트 찾기 및 데이터 채우기
    4. HTML 작성
'''
def daily_report_func(db,site_code,scrap_date=None):
    ## read list => make html => make file
    if scrap_date is None: #가장 최근 정보 읽어오기
        conn = sqlite3.connect(db)
        with conn:
            cur = conn.cursor()
            #cur.execute('select max(scrap_date) from airfare_scraped_data')
            cur.execute(latest_scrapdate_query(site_code))
            scrap_date = cur.fetchone()[0]

    param = copy.deepcopy(site_code);
    param['scrap_date'] = scrap_date
    query = report_query(param)
    query = "select airline, dpt_date, dpt, arr, flt, dpt_time, fare from(" +\
        query +\
        ") order by airline,dpt_date"

        
    report_list = []
    conn = sqlite3.connect(db)
    with conn:
        cur = conn.cursor()
        cur.execute(query)
        rows = cur.fetchall()
        for row in rows:
            ## 포맷 : 항공사,날짜,출발지,도착지,편명,출발시간,판매최저값
            report_list.append(row)
    if len(report_list) <= 0:
        return None
    ## 0. DataFrame 생성, fare의 '' 값 0 으로 보정
    columns = ['airline', 'dpt_date', 'dpt', 'arr', 'flt', 'dpt_time', 'fare']
    df = pd.DataFrame(report_list,columns=columns)
    ## fare 컬럼에 빈 문자를 포함하는 경우 0 처리
    if df['fare'].dtype == object:
        df.ix[df['fare'] == '','fare'] = 0 ## df[df == ''] = 0
    ## 1. 최소값 데이터 필터
    dfmin = df.groupby(['airline','dpt_date','dpt','arr','flt','dpt_time']).agg(agg_min)
    dfmin = dfmin.reset_index()
    ## 2. ZE 기준 경쟁 플라이트 정보 처리
    dfze = dfmin[dfmin['airline'] == 'ZE']
    dfze = dfze.reset_index()[dfmin.columns]
    ## df에 dpt_min(dpt_time을 분으로 계산한 필드) 추가 - 경쟁 플라이트 계산용
    dfmin['dpt_min'] = dfmin['dpt_time'].map(lambda x:int(x[:2])*60 + int(x[2:]))
    ## 3. 경쟁 플라이트 정보 찾기
    airlines = set(dfmin['airline'].unique()) - set(['ZE'])
    dpt_dates = dfze['dpt_date'].unique()
    cal_min = lambda x:int(x[:2])*60+int(x[2:])
    ## 값을 채울 컬럼 생성
    for airline in  airlines:
        dfze[airline+'-info'] = ''
        dfze[airline+'-fare'] = 0
    print("checking other airline's competitive fare")
    for dpt_date in dpt_dates:
        zeobj = dfze[dfze['dpt_date']==dpt_date]
        print('-- calculating on {} - data length : {}'.format(dpt_date,len(zeobj)),end=' = ')
        for airline in airlines:
            dfcmp = dfmin[(dfmin['airline'] == airline) & (dfmin['dpt_date'] == dpt_date)]
            if len(dfcmp) == 0:
                continue
            print(airline,end='/')
            for i in zeobj.index:
                dfcmp['dpt_cmp'] = abs(dfcmp['dpt_min'] - cal_min(zeobj.ix[i,'dpt_time']))
                argmin = dfcmp['dpt_cmp'].argmin()
                dfze.ix[i,airline+'-info'] = dfcmp.ix[argmin,'airline'] +\
                        '-' + dfcmp.ix[argmin,'flt'] + '-' + dfcmp.ix[argmin,'dpt_time']
                dfze.ix[i,airline+'-fare'] = dfcmp.ix[argmin,'fare']
        print('')
    ## 로드팩터 처리용 리스트
    lf_below_list = get_lf_below_list(site_code['dpt'],site_code['arr'])
    html = daily_report_html(dfze[dfze.columns[1:]],REPORT_TEMPLATE_FILE[site_code['code']])
    print('end checking')
    return html.replace('{lf_datas}',str(lf_below_list))
## 템플릿 이용 HTML 파일 작성
def daily_report_html(df,template):#sitecodes,dpt,arr,scrap_date):
    ## 검색조건 만들기
    title = 'daily report'
    column_list = list(df.columns)
    data_list = [list(row) for row in df.values]
    column_html = make_grid_th(column_list)
    
    formats = [[0,lambda d:d[:4]+'-'+d[4:6]+'-'+d[6:]],[4,time_format],[5,num_format]]
    ## 경쟁사 리스트에 따른 포맷 처리를 위해 컬럼수를 이용하여 처리, 6번째 인덱스의 컬럼부터가 추가된 부분
    formats.extend([[7+2*i,num_format] for i in range(len(column_list[7::2]))])
    ## data_list가 column_list 보다 적을 경우 data_list의 사이즈 조정 빈값으로 채우기
    data_list = [td+[''for _ in range(len(column_list)-len(td))] for td in data_list]
    grid_html = make_grid_td(list_formatter(data_list,formats))
    with open(template,encoding='utf-8') as fp:
        template = fp.read()
    ## html 생성
    return template.replace('{title}',title).replace('{columns}',column_html).replace('{grid_datas}',grid_html).replace('\n','')
## 최소값 처리 그룹화 함수
## 0이상인 값에 대해 최대값이 0이하이면 0, 아니면 0을 제외한 최소값 리턴
def agg_min(arr):
    narr = arr.values
    narr = narr[narr > 0]
    if len(narr) <= 0:
        return 0
    else:
        return sorted(narr)[0]
## 로드팩터 60% 이하 리스트 작성용, 나비테어 리포트를 통해 생성된 csv 파일 이용
## 해당 폴드에 파일을 찾아 처리후 파일들 앞에 날짜 붙여 join_ok 폴더로 이동 처리
## 파일 생성 부분 혹은 이외 부분에 처리 기능 추가 필요
def get_lf_below_list(dpt,arr):
    file_dir = JOIN_DIR #'join_data'
    files = get_files(file_dir)
    #files = get_files(file_dir,datetime.datetime.today().strftime('%Y%m%d'))
    lf_file = None
    for f in files:
        f_info = f.replace(' ','_').split('_')
        #f_info = f.split('_')
        if dpt in f_info and arr in f_info:
            lf_file = file_dir + '/' + f
    if lf_file is None:
        return '[]'
    try:
        lf_df = pd.read_csv(lf_file,usecols=['Flight Number','Origin','Destination','Departure Date','Flight LF % (Lid)'])
    except FileNotFoundError as fe:
        return '[]'
    lf_df.columns = ['flt','dpt','arr','date','lf']
    lf_df[['flt','date','lf']] = lf_df[['flt','date','lf']].apply(lambda x:[str(x[0]),x[1][:10],str(int(x[2]))],axis=1)
    return [list(v) for v in lf_df.values]
## join data 처리후 이동 처리, 이동시 파일 명에 날짜 추가
def move_join_files():
    src = JOIN_DIR
    dst = JOIN_OK_DIR
    print('start moving join data files')
    join_files = get_files(src)
    print('join files : ',len(join_files))
    for f in join_files:
        ## 파일 이동 및 이름 변경
        move_file(os.path.join(src, f),dst)
        os.rename(os.path.join(dst,f),os.path.join(dst,datetime.datetime.today().strftime('%Y%m%d_')+f))
    print('end moving join files')

In [4]:
## 환경 변수 체크하여 메일 서비스 처리 함수
'''
    0. 발송 파일 저장을 위한 폴더 생성
    1. 세팅 정보 체크 및 반복
        - 해당 함수 이용 파일 내용 생성
        - 파일 작성
        - 폴더에 파일 저장
    2. 세팅 정보에 맞는 파일 발송
'''
## mail service 대상 파일 생성
def make_airfare_report_file(site_code, save_dir=None):
    log("start making files for email service")
    ## 1. 폴더 생성
    parent_dir = "html"
    db = 'airfare_scraped_data.db'
    today = datetime.datetime.today().strftime("%Y-%m-%d")
    ## 2. 내용 생성 및 파일 생성
    if save_dir is None:
        save_dir = init_mail_folder(parent_dir)
    print("start making html files")
    for code in site_code:
        print("code : ",code)
        func = eval(code['func'])
        print("function : ",func)
        html = func(db,code)
        if html is None:
            continue
        html_file = save_dir+"/"+ html_filename(code)
        print(html_file, len(html))
        save_raw_data(html_file,html)
    log("end making files for email service")
    return save_dir
## 메일 서비스 대상자에게 해당 폴더의 파일 발송 처리
def airfare_mailing_sender(mailing_list, mail_folder, mail_msg):
    ## 3. 메일 발송
    ## 메시지 작성
    subject = '일일 온라인 가격 비교 서비스'
    mail_html = make_newletter_html(mail_msg)
    file_list = get_files(mail_folder)
    for target,auths in mailing_list.items():
        attachs = get_auth_file(auths,mail_folder,file_list)
        if len(attachs) <= 0:
            continue
        log("sending email",target)
        email_sender(target, subject, mail_html, attachs=attachs)
    log("end email service")
## Auth 코드를 이용하여 발송 대상 파일 리스트 찾아오기
def get_auth_file(auths, pdir, file_list):
    attachs = []
    for f in file_list:
        file_auth = f.split("_")[0]
        ## 기본 권한과 국내선/국제선 전체 권한 체크
        if file_auth[:1]+'00' in auths or file_auth in auths:
            attachs.append(pdir+"/"+f)
    return attachs

## auth : 국내국제구분(0,1) + 실시간구분(0,1)+번호
## 인터파크 국내선
ip_gmpcju60 = { "code":"FareReport","func":"fare_report_func","days":"60","isairline":False,"auth":"001",
             "sitecodes":['IP'],"airline":['7C','LJ','TW','ZE'],"dpt":"GMP","arr":"CJU"}
ip_gmpcju21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":False,"auth":"002",
             "sitecodes":['IP'],"airline":['7C','LJ','TW','ZE'],"dpt":"GMP","arr":"CJU"}
ip_cjugmp21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":False,"auth":"002",
             "sitecodes":['IP'],"airline":['7C','LJ','TW','ZE'],"dpt":"CJU","arr":"GMP"}
ip_gmppus21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":False,"auth":"003",
             "sitecodes":['IP'],"airline":['7C','BX','OZ','ZE'],"dpt":"GMP","arr":"PUS"}
ip_pusgmp21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":False,"auth":"003",
             "sitecodes":['IP'],"airline":['7C','BX','OZ','ZE'],"dpt":"PUS","arr":"GMP"}
ip_puscju21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":False,"auth":"004",
             "sitecodes":['IP'],"airline":['7C','LJ','OZ','KE','ZE'],"dpt":"PUS","arr":"CJU"}
ip_cjupus21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":False,"auth":"004",
             "sitecodes":['IP'],"airline":['7C','LJ','OZ','KE','ZE'],"dpt":"CJU","arr":"PUS"}
ip_cjjcju21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":False,"auth":"005",
             "sitecodes":['IP'],"airline":['7C','LJ','OZ','KE','ZE'],"dpt":"CJJ","arr":"CJU"}
ip_cjucjj21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":False,"auth":"005",
             "sitecodes":['IP'],"airline":['7C','LJ','OZ','KE','ZE'],"dpt":"CJU","arr":"CJJ"}
ip_kubcju21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":False,"auth":"006",
             "sitecodes":['IP'],"airline":['KE','ZE'],"dpt":"KUV","arr":"CJU"}
ip_cjukuv21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":False,"auth":"006",
             "sitecodes":['IP'],"airline":['KE','ZE'],"dpt":"CJU","arr":"KUV"}
## 인터파크 국제선
ip_icnnrt60 = { "code":"FareReport","func":"fare_report_func","days":"60","isairline":False,"auth":"101",
             "sitecodes":['IP'],"airline":['7C','LJ','TW','ZE'],"dpt":"ICN", "arr":"NRT"}
ip_icnnrt21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":False,"auth":"102",
             "sitecodes":['IP'],"airline":['7C','LJ','TW','ZE'],"dpt":"ICN","arr":"NRT"}
ip_nrticn21 = { "code":"FareReport","func":"fare_report_func","days":"21","isairline":False,"auth":"102",
             "sitecodes":['IP'],"airline":['7C','LJ','TW'],"dpt":"NRT","arr":"ICN"}
ip_icnbkk21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":False,"auth":"103",
             "sitecodes":['IP'],"airline":['7C','LJ','TW','ZE'],"dpt":"ICN","arr":"BKK"}
ip_bkkicn21 = { "code":"FareReport","func":"fare_report_func","days":"21","isairline":False,"auth":"103",
             "sitecodes":['IP'],"airline":['7C','LJ','TW'],"dpt":"BKK","arr":"ICN"}
ip_icnfuk21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":False,"auth":"104",
             "sitecodes":['IP'],"airline":['7C','LJ','TW','ZE'],"dpt":"ICN","arr":"FUK"}
## 항공사 국제선 현지 출발 21일
ar_nrticn21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":True,"auth":"102",
             "sitecodes":['7C','LJ','TW','ZE'],"airline":['7C','LJ','TW','ZE'],"dpt":"NRT","arr":"ICN"}
ar_bkkicn21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":True,"auth":"103",
             "sitecodes":['7C','LJ','TW','ZE'],"airline":['7C','LJ','TW','ZE'],"dpt":"BKK","arr":"ICN"}
ar_fukicn21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":True,"auth":"104",
             "sitecodes":['7C','LJ','TW','ZE'],"airline":['7C','LJ','TW','ZE'],"dpt":"FUK","arr":"ICN"}
## 항공사 김포제주 60일
ar_gmpcju60 = { "code":"FareReport","func":"fare_report_func","days":"60","isairline":True,"auth":"011",
             "sitecodes":['7C','LJ','TW','BX','ZE'],"airline":['7C','LJ','TW','BX','ZE'],"dpt":"GMP","arr":"CJU"}

## 치토세 왕복
ar_icncts21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":True,"auth":"105",
             "sitecodes":['7C','LJ','TW','ZE','KE'],"airline":['7C','LJ','TW','ZE','KE'],"dpt":"ICN","arr":"CTS"}
ar_ctsicn21 = { "code":"DailyReport","func":"daily_report_func","days":"21","isairline":True,"auth":"105",
             "sitecodes":['7C','LJ','TW','ZE','KE'],"airline":['7C','LJ','TW','ZE','KE'],"dpt":"CTS","arr":"ICN"}


## 하루 세번 생성
## 생성 타입을 받아 해당 파일 생성(기간, 국내/국제구분)
## 기간 None: 모두, 1 : 21, 2 : 60, 국내/국제 : 0, 1
def make_report_file_by_type(period=None,dom_int=None,save_dir=None):
    conds_dom_60 = [ip_gmpcju60]
    conds_int_60 = [ip_icnnrt60]
    conds_dom_21 = [ip_gmpcju21, ip_cjugmp21, ip_gmppus21, ip_pusgmp21,
                        ip_puscju21, ip_cjupus21, ip_cjjcju21, ip_cjucjj21, ip_kubcju21, ip_cjukuv21]
    conds_int_21 = [ip_icnnrt21, ar_nrticn21, ip_icnbkk21, ar_bkkicn21, ip_icnfuk21, ar_fukicn21,
                   ar_icncts21, ar_ctsicn21]

    conds = []
    if period is None and dom_int is None:
        conds = conds_dom_60 + conds_int_60 + conds_dom_21 + conds_int_21
    elif period is not None:
        if period == 1:
            if dom_int == 0:
                conds = conds_dom_21
            elif dom_int == 1:
                conds = conds_int_21
            else:
                conds = conds_dom_21 + conds_int_21
        if period == 2:
            if dom_int == 0:
                conds = conds_dom_60
            elif dom_int == 1:
                conds = conds_int_60
            else:
                conds = conds_dom_60 + conds_int_60
    elif dom_int == 0:
        conds = conds_dom_21 + conds_dom_60
    elif dom_int == 1:
        conds = conds_int_21 + conds_int_60
    return make_airfare_report_file(conds,save_dir=save_dir)


In [5]:
### 하루 한번 작업 - 출근후
## 60 데이터 작업
save_dir = make_report_file_by_type(period=2)
## 21 데이터 작업
save_dir = make_report_file_by_type(period=1,save_dir=save_dir)

2017-08-25 09:21:19,139 root     DEBUG    start making files for email service


start making html files
code :  {'days': '60', 'dpt': 'GMP', 'isairline': False, 'airline': ['7C', 'LJ', 'TW', 'ZE'], 'auth': '001', 'code': 'FareReport', 'sitecodes': ['IP'], 'arr': 'CJU', 'func': 'fare_report_func'}
function :  <function fare_report_func at 0x0000000007857BF8>
query :  select max(scrap_date) from airfare_scraped_data where scrap_site = "IP"
query :  select airline, flt, dpt_date, dpt, arr, dpt_time, arr_time, fare, tax1, tax2, seat
        from airfare_scraped_data
        where scrap_date = '2017082508' and scrap_site in ('IP')
        and dpt='GMP' and arr='CJU'
        and dpt_date <= strftime('%Y%m%d','now','+60 day')
        and airline in ('7C','LJ','TW','ZE')
html/20170825_1/001_FareReport_B_20170825_GMPCJU_60_1503620479.html 48342
code :  {'days': '60', 'dpt': 'ICN', 'isairline': False, 'airline': ['7C', 'LJ', 'TW', 'ZE'], 'auth': '101', 'code': 'FareReport', 'sitecodes': ['IP'], 'arr': 'NRT', 'func': 'fare_report_func'}
function :  <function fare_report_func

2017-08-25 09:21:19,938 root     DEBUG    end making files for email service
2017-08-25 09:21:19,940 root     DEBUG    start making files for email service


query :  select airline, flt, dpt_date, dpt, arr, dpt_time, arr_time, fare, tax1, tax2, seat
        from airfare_scraped_data
        where scrap_date = '2017082508' and scrap_site in ('IP')
        and dpt='ICN' and arr='NRT'
        and dpt_date <= strftime('%Y%m%d','now','+60 day')
        and airline in ('7C','LJ','TW','ZE')
html/20170825_1/101_FareReport_B_20170825_ICNNRT_60_1503620479.html 49848
start making html files
code :  {'days': '21', 'dpt': 'GMP', 'isairline': False, 'airline': ['7C', 'LJ', 'TW', 'ZE'], 'auth': '002', 'code': 'DailyReport', 'sitecodes': ['IP'], 'arr': 'CJU', 'func': 'daily_report_func'}
function :  <function daily_report_func at 0x0000000007857840>
query :  select max(scrap_date) from airfare_scraped_data where scrap_site = "{'days': '21', 'dpt': 'GMP', 'isairline': False, 'airline': ['7C', 'LJ', 'TW', 'ZE'], 'auth': '002', 'code': 'DailyReport', 'sitecodes': ['IP'], 'arr': 'CJU', 'func': 'daily_report_func'}"
query :  select airline, flt, dpt_date, dpt,

d:\python\python35\lib\site-packages\ipykernel\__main__.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


TW/7C/
-- calculating on 20170827 - data length : 19 = LJ/TW/7C/
-- calculating on 20170828 - data length : 21 = LJ/TW/7C/
-- calculating on 20170829 - data length : 20 = LJ/TW/7C/
-- calculating on 20170830 - data length : 22 = LJ/TW/7C/
-- calculating on 20170831 - data length : 19 = LJ/TW/7C/
-- calculating on 20170901 - data length : 20 = LJ/TW/7C/
-- calculating on 20170902 - data length : 19 = LJ/TW/7C/
-- calculating on 20170903 - data length : 19 = LJ/TW/7C/
-- calculating on 20170904 - data length : 21 = LJ/TW/7C/
-- calculating on 20170905 - data length : 20 = LJ/TW/7C/
-- calculating on 20170906 - data length : 22 = LJ/TW/7C/
-- calculating on 20170907 - data length : 20 = LJ/TW/7C/
-- calculating on 20170908 - data length : 20 = LJ/TW/7C/
-- calculating on 20170909 - data length : 19 = LJ/TW/7C/
-- calculating on 20170910 - data length : 19 = LJ/TW/7C/
-- calculating on 20170911 - data length : 21 = LJ/TW/7C/
-- calculating on 20170912 - data length : 20 = LJ/TW/7C/
-- calc

2017-08-25 09:24:48,163 root     DEBUG    end making files for email service



end checking
html/20170825_1/105_DailyReport_A_20170825_CTSICN_21_1503620688.html 27602


In [ ]:
## 하루 두번 작업 - 10시, 14시
## 21 데이터 작업
save_dir = None
save_dir = make_report_file_by_type(period=1,save_dir=save_dir)

In [6]:
test_list = { ## 수신자 리스트, REPORT_SITE_CODE 의 auth 와 연계
    "관리자<park363@eastarjet.com>":["000","100"],
}

## 코드 00 - 국내선 전체 수신, 10 - 국제선 전체 수신
target_list = { ## 수신자 리스트, REPORT_SITE_CODE 의 auth 와 연계
    "관리자<park363@eastarjet.com>":["100"],
    '우성주<topatsee@eastarjet.com>':["000","100"],
    '조용현<ycho1211@eastarjet.com>':["000","100"],
    '염희석<hsyoum@eastarjet.com>':["000","100"],
    '김희정<amykim@eastarjet.com>':["000","100"],
    '정희진<tokkaipi@eastarjet.com>':["000","100"],
    '안병준<bjahn@eastarjet.com>':["000","100"],
    '장혜란<jhyeran@eastarjet.com>':["000","100"],
    '이동현<ldh1@eastarjet.com>':["000"],
    '이재은<leeje@eastarjet.com>':["000","100"],
    '임서윤<lhj01@eastarjet.com>':["000"],
    '황혜진<hhj@eastarjet.com>':["000"],
    '이석규<sklee@eastarjet.com>':["100"],
    '안경미<km9980@eastarjet.com>':["100"],
    '권재혁<jhkwon@eastarjet.com>':["000","100"],
    '김혜신<khs@eastarjet.com>':["000"],
    '김이현<kuvkyh@eastarjet.com>':["000"],
    '김지은<jekim@eastarjet.com>':["000"],
    '김현지<hyunjiop@eastarjet.com>':["000"],
    '김병준<zekbj@eastarjet.com>':["000","100"],
    '정찬왕<jcw@eastarjet.com>':["000","100"],
    '정미금<loki@eastarjet.com>':["100"],
    '한주연<jyhan@eastarjet.com>':["100"],
    '변지숙<jsturtle@eastarjet.com>':["000","100"],
}

## 메일 내용
mail_msg = {
    'title1' : 'RMS 뉴스 레터 - 온라인 경쟁사 가격 비교[노선 추가]',
    'title2' : '21일 플라이트별 가격 비교 / 60일 가격 트랜드 비교',
    'date' : datetime.datetime.today().strftime("%Y-%m-%d"),
    'auth' : '관리자<park363@eastarjet.com>',
    'contents' : '''
    <p>온라인 상의 경쟁 항공사 가격 비교 입니다.</p>
    <p>수신자에 따라 2가지 종류의 파일이 발송 됩니다.</p>
    <p>DailyReport로 시작하는 파일이 21일 플라이트별 가격 비교 파일이며</p>
    <p>FareReport로 시작하는 파일이 60일 가격 트랜드 비교 파일입니다.</p>
    <p>A는 항공사, B는 실시간 여행사를 나타내며 이후 내용은 의미를 유추하실수 있을 것 입니다.</p>
    <p>배치 작업은 하루 세번 진행 됩니다. 8시~9시, 10시~11시, 14시~15시, 작업이 종료 되면 메일이 발송됩니다.</p>
    <p>작업중 예기치 못한 상황이 발생되면 추가 체크 작업이 진행되어 조금 지체될 수도 있습니다.</p>
    <p>60일 데이터는 국내선 김포 제주, 국제선 인천 나리타 노선을 하루 한번</p>
    <p>21일 데이터는 국내선 전노선 왕복, 국제선 인천 나리타/방콕 편도를 기준으로 하루 세번 진행됩니다..</p>
    <p>점진적으로 확대 적용 할 수 있도록 하겠습니다.</p>
    <br><br><p style="font-weight:bold;">[노선 추가]</p>
    <p>치토세 노선 추가 하였습니다.(김병준 차장)</p>
    <p>경쟁 항공사의 홈페이지를 통해 21일치 데이터를 이용하여 추가하였습니다.</p>
    <br><br><p style="font-weight:bold;">[요청 사항]</p>
    <p>노선을 추가하거나 기타 요청 사항이 있을 경우 김희정 과장과 상의 하시기 바랍니다.</p>
    <p>요청 사항은 적극 반영 하도록 하겠습니다.</p>
    <br><br><p style="font-weight:bold;">[주의 사항]</p>
    <p>HTML 파일을 첨부하여 아웃룩에서는 첨부파일이 확인 안되는 경우가 있습니다.</p>
    <p>가능하면 그룹웨어 메일에서 직접 확인하시기 바랍니다.</p>
    <p>그리고 HTML5 SVG 기술을 이용하여 차트를 구현하였습니다. 가능하면 크롬브라우저를 사용하기를 권장드립니다.</p>
    <br><br><p style="font-weight:bold;">[기타 사항]</p>
    <p>동남아 항공사 개발 진행은 장비 세팅 및 추가 개발 완료후 진행 하겠습니다.</p>
    <p>당분간 1일 1회 발송 됩니다. 참고하시고 추가 데이터가 필요하신분은 요청해 주시면 처리 하도록 하겠습니다.</p>
    '''
}

## 폴더 생성된 경우 메일발송
receiver = test_list
receiver = target_list
folder = save_dir 
#folder = 'html/20170712_1'
airfare_mailing_sender(receiver,folder,mail_msg)

2017-08-25 09:29:59,498 root     DEBUG    receiver :: 안병준<bjahn@eastarjet.com> ::  ## email - :: 일일 온라인 가격 비교 서비스


$$ Check : sending email


2017-08-25 09:30:01,827 root     DEBUG    receiver :: 김혜신<khs@eastarjet.com> ::  ## email - :: 일일 온라인 가격 비교 서비스


$$ Check : sending email


2017-08-25 09:30:02,672 root     DEBUG    receiver :: 염희석<hsyoum@eastarjet.com> ::  ## email - :: 일일 온라인 가격 비교 서비스


...

2017-08-25 09:30:25,997 root     DEBUG    receiver :: 안경미<km9980@eastarjet.com> ::  ## email - :: 일일 온라인 가격 비교 서비스


$$ Check : sending email


2017-08-25 09:30:27,222 root     DEBUG    receiver :: 조용현<ycho1211@eastarjet.com> ::  ## email - :: 일일 온라인 가격 비교 서비스


$$ Check : sending email


2017-08-25 09:30:28,235 root     DEBUG    end email service


In [ ]:
## 파일 별도 생성시
save_dir = 'html/20170720_1'
make_airfare_report_file([ip_icnfuk21,ar_fukicn21],save_dir = save_dir)

In [ ]:
## 처리후 join 파일 이동 처리
move_join_files()